In [ ]:
#!pip install openai-whisper pandas tqdm
#!pip install ffmpeg-python

# Language Detection Pipeline

## Overview
This Jupyter notebook implements a language detection pipeline for YouTube videos using OpenAI's Whisper model. It processes video files to detect their primary spoken language and confidence scores, with SQLite-based progress tracking.

### Key Features
- Language detection using Whisper-large model
- Progress tracking with SQLite database
- Fault-tolerant processing with timeouts
- Batch processing capabilities
- Confidence score filtering

### Prerequisites


In [ ]:
import os
import random
import subprocess
import whisper
import pandas as pd
import sqlite3
from IPython.display import clear_output



### Process Flow
1. Initialize SQLite database for tracking
2. Process videos in batches:
   - Extract first 30 seconds of audio
   - Detect language using Whisper
   - Store results with confidence scores
3. Filter results for high-confidence detections (≥0.98)
4. Export results to CSV files:
   - `Detected_Language.csv`: All detections
   - `Detected_Language_Confident.csv`: High-confidence only

### Configuration
- Input: MP4 video files
- Output: CSV with video IDs, detected languages, and confidence scores
- Database: SQLite for process tracking
- Model: Whisper-large for accurate language detection

### Usage
The pipeline processes videos from a specified input folder and maintains processing state, allowing for interrupted runs to be resumed.

In [ ]:
DB_PATH = "video_log.db"

# Ensure the database and table exist
def initialize_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS videos (
                    video_id TEXT PRIMARY KEY,
                    status INTEGER DEFAULT 0,
                    timestamp REAL
                )''')
    conn.commit()
    conn.close()

def add_videos_to_db(input_folder):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    
    mp4_files = [f for f in os.listdir(input_folder) if f.endswith(".mp4")]
    
    for filename in mp4_files:
        video_id = os.path.splitext(filename)[0]
        
        # Try inserting, ignore if video already exists
        c.execute("INSERT OR IGNORE INTO videos (video_id, status, timestamp) VALUES (?, 0, NULL)", (video_id,))
    
    conn.commit()
    conn.close()

In [ ]:
def get_next_video():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    
    # Get an unprocessed video (`status=0`) OR a stale `status=1` (older than 10 minutes)
    c.execute("""
        SELECT video_id FROM videos 
        WHERE status = 0 
        OR (status = 1 AND timestamp < ?) 
        ORDER BY status ASC, timestamp ASC 
        LIMIT 1
    """, (time.time() - 600,))  # 10 minutes timeout
    
    row = c.fetchone()
    if row:
        video_id = row[0]
        c.execute("UPDATE videos SET status = 1, timestamp = ? WHERE video_id = ?", (time.time(), video_id))
        conn.commit()
        conn.close()
        return video_id
    conn.close()
    return None

In [ ]:
def process_video(video_id, input_folder, output_path, model, s):
    video_path = os.path.join(input_folder, f"{video_id}.mp4")
    trimmed_audio_path = os.path.join(input_folder, f"trimmed_{video_id}.wav")

    # Trim first 30 seconds using ffmpeg
    subprocess.run([
        "ffmpeg", "-i", video_path, "-t", s, "-q:a", "0", "-map", "a", trimmed_audio_path, "-y"
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Load and process the audio
    audio = whisper.load_audio(trimmed_audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio=audio, n_mels=128).to(model.device)

    # Detect language
    _, probs = model.detect_language(mel) 
    detected_language = max(probs, key=probs.get)
    confidence = probs[detected_language]

    # Remove temp file
    os.remove(trimmed_audio_path)

    # Save result to a CSV file (append mode)
    df = pd.DataFrame([{"Video ID": video_id, "language": detected_language, "confidence": confidence}])
    df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

    # Mark video as completed
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("UPDATE videos SET status = 2 WHERE video_id = ?", (video_id,))
    conn.commit()
    conn.close()

    print(f"Processed {video_id}: {detected_language} ({confidence:.3f})")


In [ ]:
def update_print(progress): 
    """Clears Jupyter output and prints the updated status of videos."""
    clear_output(wait=True)  # This will clear only the last output instead of flashing

    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    
    c.execute("SELECT COUNT(*) FROM videos")
    total_videos = c.fetchone()[0]
    
    c.execute("SELECT COUNT(*) FROM videos WHERE status = 2")
    completed_videos = c.fetchone()[0]
    
    c.execute("SELECT COUNT(*) FROM videos WHERE status = 0")
    unprocessed_videos = c.fetchone()[0]
    
    c.execute("SELECT COUNT(*) FROM videos WHERE status = 1")
    in_progress_videos = c.fetchone()[0]
    
    conn.close()

    print(f"📌 Total videos in database:    {total_videos}")
    print(f"✅ Completed videos:            {completed_videos} [+{progress}] ({completed_videos/total_videos * 100:.1f}%)")
    print(f"⏳ Still unprocessed:           {unprocessed_videos} ({unprocessed_videos/total_videos * 100:.1f}%)")
    print(f"⚠️  In progress (interrupted):  {in_progress_videos} ({in_progress_videos/total_videos * 100:.1f}%)")

    return total_videos, completed_videos

In [ ]:
def process_all_videos(input_folder, output_path, model, seconds="30"):
    initialize_db()
    add_videos_to_db(input_folder)

    progress = 0
    while True:
        video_id = get_next_video()
        if not video_id:
            print("\n✅ No videos left to process. Exiting...\n")
            break
            
        total_videos, completed_videos = update_print(progress)  # Clears screen & updates progress
        
        print(f"\n🚀 Processing video: {video_id} ...")
        process_video(video_id, input_folder, output_path, model=model, s=seconds)
        progress += 1


In [ ]:
# setup
input_folder = "../../../SDU_data/YouTube_Downloader/Downloads"
output_path = "Detected_Language.csv"
print("Loading Whisper-Large...")
model = whisper.load_model("large")

In [ ]:
input_folder = "../../../SDU_data/YouTube_Downloader/Downloads"

In [ ]:
# run master program:
process_all_videos(input_folder, output_path, model=model)

---

In [ ]:
def reset_incomplete_videos():
    """Reset all videos that were started (status=1) but not finished (status=2) back to unprocessed (status=0)."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    
    # Reset any videos stuck in "in progress" state
    c.execute("UPDATE videos SET status = 0 WHERE status = 1")
    conn.commit()
    
    conn.close()
    print("🔄 Reset all incomplete videos back to unprocessed (status = 0).")


In [ ]:
reset_incomplete_videos()

---

In [ ]:
update_print(0)

In [ ]:
import sqlite3

conn = sqlite3.connect("video_log.db")
c = conn.cursor()

# View all records
c.execute("SELECT * FROM videos WHERE status != 2")
rows = c.fetchall()
for row in rows:
    print(row)

conn.close()

In [ ]:
df = pd.read_csv("Detected_Language.csv")
csv_file = "../../YouTube_Downloader/Participants_with_20_videos.csv"
import ast
videos = pd.read_csv(csv_file, converters={"videos": ast.literal_eval})
video_ids = videos['videos'].explode().tolist()
df_filtered = df[df['Video ID'].isin(video_ids)]
df_filtered.to_csv("Detected_Language.csv", index=False)

In [ ]:
df_filtered

In [ ]:
df_filtered['language'].value_counts()

In [ ]:
df_confident = df_filtered[df_filtered['confidence']>= 0.98]
df_confident.to_csv("Detected_Language_Confident.csv", index=False)
df_confident

In [ ]:
df_confident['language'].value_counts()